In [3]:
# Reference: https://jupyterbook.org/interactive/hiding.html
# Use {hide, remove}-{input, output, cell} tags to hiding content

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import myst_nb

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

def display_df(df, rows=pd.options.display.max_rows,
               cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

(ch:files_size)=
# File Size


Computers have finite limits on computing power. You have likely
encountered these limits firsthand if your computer has slowed down from having
too many applications open at once. We want to make sure that we do not
exceed the computer's limits while working with data, and we might examine a file differently, depending on its size. If we know that our data set is relatively small, then a text editor or a spreadsheet can be convenient to look at the data. On the other hand, for large datasets, a more programmatic exploration or even distributed computing tools may be needed.

In many situations, we analyze datasets downloaded from the Internet. These
files reside on the computer's **disk storage**. In order to use Python to
explore and manipulate the data, we need to read the data into the computer's
**memory**, also known as random access memory (RAM). All Python code requires
the use of RAM, no matter how short the code is.

A computer's RAM is typically much smaller than a computer's disk storage. For
example, one computer model released in 2018 had 32 times more disk storage
than RAM.  Unfortunately, this means that data files can often be much bigger
than what is feasible to read into memory. 
Both disk storage and RAM capacity are measured in terms of **bytes**. Roughly
speaking, each character in a text file adds one byte to the file's size. For
example, the legends.csv file has 120 characters and takes up 120 bytes of
disk space.


Of course, many of the datasets we work with today contain many characters. To
succinctly describe the sizes of larger files, we use the prefixes as described
in the following {numref}`byte-prefixes`.

:::{table} Prefixes for common filesizes.
:name: byte-prefixes

| Multiple | Notation | Number of Bytes |
| -------- | -------- | --------------- |
| Kibibyte | KiB      | 1024    |
| Mebibyte | MiB      | 1024²   |
| Gibibyte | GiB      | 1024³   |
| Tebibyte | TiB      | 1024⁴   |
| Pebibyte | PiB      | 1024⁵   |

:::


For example, a file containing 52428800 characters takes up 52428800 bytes = 50
mebibytes = 50 MiB on disk.

**Why use multiples of 1024 instead of simple multiples of 1000 for these
prefixes?** This is a historical result of the fact that most computers
use a binary number scheme where powers of 2 are simpler to represent. You will
also see the typical SI prefixes used to describe size---kilobytes, megabytes,
and gigabytes, for example. Unfortunately, these prefixes are used
inconsistently. Sometimes a kilobyte refers to 1000 bytes; other times, a
kilobyte refers to 1024 bytes. To avoid confusion, we will stick to kibi-,
mebi-, and gibibytes which clearly represent multiples of 1024.

**When is it safe to read in a file?** Many computers have much more disk
storage than available memory. It is not uncommon to have a data file happily
stored on a computer that will overflow the computer's memory if we attempt to
manipulate it with a program, including Python programs. We often begin our
data work by making sure the files we are of manageable size. To accomplish
this, we can use the built-in `os` library.

In [18]:
from pathlib import Path
import os

kib = 1024
line = '{:<25} {}'.format

print(line('File', 'Size (KiB)'))
for filepath in Path('data').glob('*'):
    size = os.path.getsize(filepath)
    print(line(str(filepath), np.round(size / kib)))

File                      Size (KiB)
data/inspections.csv      455.0
data/co2_mm_mlo.txt       50.0
data/violations.csv       3639.0
data/DAWN-Data.txt        273531.0
data/legend.csv           0.0
data/businesses.csv       645.0


We see that the `businesses.csv` file takes up 645 KiB on disk, making it well
within the memory capacities of most systems. Although the `violations.csv`
file takes up 3.6 MiB of disk storage, most machines can easily read it into a
Pandas dataframe too. But `DAWN-Data.txt`, which contains the DAWN survey data,
is much larger.

The DAWN file takes up nearly 270 MiB of disk storage, and while some computers
can work with this file in memory, it can slow down other systems. To make
this data more manageable in Python, we can load in a subset of the columns
rather than all of them.

**Folder Sizes.** Sometimes we are interested in the total size of a folder
instead of the size of individual files. For example, if we have one file of
inspections for each month in a year, we might like to see whether we can
combine all the data into a single data frame.

In [19]:
mib = 1024**2

total = 0
for filepath in Path('data').glob('*'):
    total += os.path.getsize(filepath) / mib

print(f'The data/ folder contains {total:.2f} MiB')

The data/ folder contains 271.80 MiB


**Memory Overhead.** As a rule of thumb, reading in a file using `pandas`
usually requires at least double the available memory as the file size. That
is, reading in a 1 GiB file will typically require at least 2 GiB of available
memory.

Note that memory is shared by all programs running on a computer, including the
operating system, web browsers, and yes, Jupyter notebook itself. A computer
with 4 GiB total RAM might have only 1 GiB available RAM with many applications
running. With 1 GiB available RAM, it is unlikely that `pandas` will be able to
read in a 1 GiB file.

## How Do We Work With Large Datasets?

What can we do if we know our data are far larger than what is feasible to
load into memory? This is a common scenario in some scientific domains like 
astronomy, where telescopes capture many large images of space. It's also
common for companies that have lots of users. The popular term "big data" 
generally refers to this scenario, when the data are large enough that even
the best computers can't read the data directly into memory.

How data scientists draw insights from large datasets is an important research
question that motivates the fields of database engineering and distributed
computing. While we won't have time to cover these fields in depth in this
book, we can provide a brief overview of basic approaches. Interested readers
can investigate these approaches further in a more specialized text.


**Subset The Data.** One simple approach is to subset the data. Rather than
loading in the entire data file, we can either select a specific part of it
(e.g. one day's worth of data), or we can randomly sample the dataset. Because
of its simplicity, we use this approach quite often in this book. The natural
downside is that this approach loses many of the benefits of using a large
dataset, like being able to study rare events.

**Use a Database System.** As we discussed in the {ref}`ch:sql` chapter,
database systems like PostgreSQL are specifically design to store large
datasets. Relational database management systems (RDBMS) let users query the
data using SQL, which can even manipulate data that are too large to fit into
memory. Because of their advantages, RDBMS's are common in research and
industry settings for data storage. One downside is that they often require a
separate server for the data that needs its own configuration. Another downside
is that SQL is less flexible in what it can compute than Python, which is
especially relevant for modeling and prediction.

**Use a Distributed Computing System.** One approach to handle complex
computations on large datasets is to use a distributed computing system like
Apache Spark or Ray. These systems let users write programs in a programming
language like Python, and can automatically run these programs across large
datasets. Because of this, these systems have great flexibility and can be used
in a variety of scenarios like modeling and prediction. Their main downside is
that they typically require a lot of work to install and configure properly,
since these systems are typically installed across many computers that need to
coordinate with each other.

## Takeaways

This section introduced common file size notation, and showed how to check file
sizes in Python. We also discussed when a file is feasible to read into memory
as a `pandas.DataFrame`, and approaches for big data that can't be read into
memory. In the next section, we'll describe command line tools available in the
shell for carrying out these same tasks. 